In [26]:
import numpy as np
import pandas as pd
import re
#정규식으로 문자를 한번에 처리하기 위해서는 파이썬의 re가 필요함

from konlpy.tag import Okt
#트위터에서 만든 문형분석기 

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

## 네이버 영화 리뷰를 활용한 감정 분석 

> 1) KoNLP
 * pip install konlpy 설치 - 한국말 조사를 떼기 위해 필요함 
 * pip install joblib 

> 2) tf-idf를 활용한 임베딩

### 데이터 불러오기

In [18]:
train_df=pd.read_table("data/ratings_train.txt")
test_df=pd.read_table("data/ratings_test.txt")

In [19]:
train_df.head(n=10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [20]:
train_df.isnull().sum()

id          0
document    5
label       0
dtype: int64

### 전처리

In [21]:
train_df = train_df.fillna(" ")
test_df = test_df.fillna(" ")

In [22]:
#정규식 사용법
#"\d+"는 하나 이상의 숫자(0부터 9까지.) +는 하나 이상
#""는 대체 문자열. 즉 숫자 부분을 제거하게 됨

re.sub(r"\d+", "", "123 456 하하하")

'  하하하'

In [23]:
 # X는 문자열이 들어있는 저 document 한 행이 x가 되는거임

train_df["document"]=train_df["document"].apply(lambda x: re.sub(r"\d+", " ", x))
test_df["document"]=test_df["document"].apply(lambda x: re.sub(r"\d+", " ", x))

### 임베딩

In [30]:
okt = Okt()

def tw_tokenizer(text):
     tokenizer_ko = okt.morphs(text)
     return tokenizer_ko

#반드시 이렇게 함수로 만들어져있어야함
#okt-이 영화 재미있어요 하면 "이" 떼내요. 너무좋아요 하면 "너무"떼내요.

In [31]:
#몇퍼센트 강도를 유지할것인가 퍼센트로 적어주삼. max_df=0.9는 단어가 문서 집합의 90%에서 나타나면 무시된다는 의미. 그 이상은 불용어 처리하겠다

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9 ) 
tfidf_vect.fit(train_df["document"])
tfidf_matrix_train = tfidf_vect.transform(train_df["document"])

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 학습

In [ ]:
lr = LogisticRegression(C=3.5, random_state=42)
lr.fit(tfidf_matrix_train, train_df["label"])

#logis는 선형자료이므로 matrix_train은 벡터화 된 아이들이기 때문에 나머지 값을 같이 해주려고 치면
#벡터 안거친 애들이 이제 lr에 들어가서는 안됨 안 맞 다 고 요
#결국에는 0에 가깝냐 1에 가깝냐 그 벡터값을 정하는거라는걸 잊으면 안됨 